In [9]:
## If you run for first time uncomment these
#!pip install elasticsearch
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import json
from scipy import spatial
from elasticsearch import Elasticsearch

In [10]:
recipes = []
with open('recipes_raw_nosource_ar.json') as json_file: #make sure the recipe json is in same dir
    data = json.load(json_file)
    for p in data:
        recipes.append(data[p])
es = Elasticsearch()
for i in range(0,len(recipes[:10000])): #i took only first 120 recipes because time and size
    res = es.index(index='recipes', doc_type='recipe', id=i, body=recipes[i])

In [87]:
def giveResults(query, last, wish,title=True,instructions=True):
    res = es.search(index="recipes", body={"query": {"bool": { "must": [{"match":{"ingredients":query} }]}},"size":500})
    presults = [] #personalized
    mscores = [] # new scores for personalized results
    results = [] #unpersonalized
    for hit in res['hits']['hits']:
        if title:
            results.append((hit['_source']['title'],hit['_score']))
        if instructions: 
            results.append((hit['_source']['instructions'],hit['_score']))
        ingr = ''
        for s in hit['_source']['ingredients']:
            ingr += s
        ldocuments = (ingr,last)
        wdocuments = (ingr,wish)
        tfidf_vectorizer=TfidfVectorizer()
        tfidf_last=tfidf_vectorizer.fit_transform(ldocuments)
        tfidf_wish=tfidf_vectorizer.fit_transform(wdocuments)
        cs = -100 * cosine_similarity(tfidf_last[0:1],tfidf_last)
        ds = 100 * cosine_similarity(tfidf_wish[0:1],tfidf_wish)
        mscores.append((hit['_source'],hit['_score']+cs[0][1]+ds[0][1]))
    mscores.sort(key=lambda t: t[1],reverse=True)
    for t in mscores:
        if title:
            presults.append((t[0]['title'],t[1]))
        if instructions:
            presults.append((t[0]['instructions'],t[1]))
    return presults, results

In [13]:
def getInput():
    query = input("Enter whatever ingredients you have in your pantry seperated by whitespaces: ")
    last = input("Enter at least 5 ingredients you ate last week seperated by whitespace: ")
    wish = input("List anything you would like to eat. You can also just skip this.")
    return query,last,wish

In [46]:
def giveRecipes(title = True, instructions = False):
    query,last,wish = getInput()
    print(query)
    personalized, standard = giveResults(query,last,wish,title,instructions)
    return personalized, standard

In [90]:
p, s = giveRecipes()

Enter whatever ingredients you have in your pantry seperated by whitespaces: rosemary thyme cumin cheese rice tuna
Enter at least 5 ingredients you ate last week seperated by whitespace: chickpeas beans lentils honey chicken
List anything you would like to eat. You can also just skip this.shrimp
rosemary thyme cumin cheese rice tuna


In [91]:
s

[('Easy Herb Rice', 11.527321),
 ('Pork Chops Over Rice', 10.18104),
 ("Uncle Josh's Famous Chicken and Rice", 9.838473),
 ('Mayo-Free Tuna Sandwich Filling', 9.44627),
 ('Cheesy Tuna Dinner', 9.356092),
 ('Classic Turkey and Rice Soup', 9.337923),
 ('Black Lemon Chicken Salad', 9.322743),
 ('Chicken Seitan', 9.322743),
 ('Dry Brined Roasted Chicken', 9.119278),
 ('Leek Tomato Dish', 9.116635),
 ('Chicken Song Soup', 9.116339),
 ('Traditional Gyros', 9.08714),
 ('Parmesan Garlic Bread', 8.9774475),
 ('Italian Herb Infused Olive Oil', 8.954964),
 ('Tomato-Rich Fish Stew', 8.881586),
 ('Italian Seasoning II', 8.875006),
 ('Chicken Seasoning Blend', 8.859494),
 ('Zing-a-Ding-Ding Spaghetti with Albacore Tuna', 8.845046),
 ('Hearty Tuna Casserole', 8.845046),
 ('Chicken Melt', 8.842445),
 ('Grilled Tuna Teriyaki', 8.788832),
 ('Pan-Roasted Marble Potatoes', 8.735151),
 ('Herb Garlic Baked Potatoes', 8.719301),
 ('Roast Pheasant', 8.719301),
 ('Spicy African Yam Soup', 8.649914),
 ('Grilled

In [92]:
p

[('Shrimp and Grits With Kielbasa', 13.501341371649376),
 ('Jambalaya I', 11.963659809705673),
 ('New Orleans Barbeque Shrimp', 11.572349421591),
 ('Easy Herb Rice', 11.527321),
 ("Sarah's Delicious Bouillabaisse", 10.132142554060472),
 ('Mayo-Free Tuna Sandwich Filling', 9.44627),
 ('Cheesy Tuna Dinner', 9.356092),
 ('Classic Turkey and Rice Soup', 9.337923),
 ('Chicken Seitan', 9.322743),
 ('Leek Tomato Dish', 9.116635),
 ('Traditional Gyros', 9.08714),
 ('Parmesan Garlic Bread', 8.9774475),
 ('Italian Herb Infused Olive Oil', 8.954964),
 ('Italian Seasoning II', 8.875006),
 ('Zing-a-Ding-Ding Spaghetti with Albacore Tuna', 8.845046),
 ('Hearty Tuna Casserole', 8.845046),
 ('Grilled Tuna Teriyaki', 8.788832),
 ('Pan-Roasted Marble Potatoes', 8.735151),
 ('Herb Garlic Baked Potatoes', 8.719301),
 ('Roast Pheasant', 8.719301),
 ('Grilled Tuna Salad', 8.642319),
 ('Grilled Salmon Steaks Italian-Style', 8.568965),
 ('Tuna Dip II', 8.537345),
 ('Pork Chop Grill', 8.495725),
 ('Slow Roaste